# Self-Extending Agents with Strands Meta-Tooling

This notebook demonstrates how to create a [self-extending agent](https://strandsagents.com/latest/documentation/docs/examples/python/meta_tooling/) using the Strands Agents framework. A self-extending agent can write and dynamically use new tools during runtime, effectively expanding its own capabilities as needed.

## What You'll Learn

- How to set up a Strands agent with meta-tooling capabilities
- How to configure the system prompt to guide tool creation
- How an agent can write Python tools and test them
- How to use hot-reloading to immediately access new tools
- How to debug and troubleshoot tool creation issues

## Prerequisites

- An Anthropic API key (Claude 4 model recommended)
- Python 3.8+ environment
- Basic understanding of Python and LLM agents

Let's begin by all the requirements:

In [ ]:
# Install the required packages if not already installed
# The [anthropic] extra includes dependencies for using the Anthropic Claude models
# colorama is added for color coding the outputs
!pip install strands-agents strands-agents-tools 'strands-agents[anthropic]' python-dotenv ipykernel colorama

## 1. Environment Setup

First, we need to set up our environment by loading environment variables from a `.env` file and installing required packages.

> 📝 **Note*: Before running this notebook, rename the `.env.example` file to `.env` in the same directory and update it with your Anthropic API key: `ANTHROPIC_API_KEY=your_key_here`. This is essential for the agent to function.

In [ ]:
# Load environment variables from the .env file
# This allows secure storage of API keys and configuration
from dotenv import load_dotenv
load_dotenv()

## 2. Import Required Libraries

Next, we import the necessary libraries:
- `strands`: The core framework for creating agents
- `strands_tools`: Provides tools like `editor` for file operations and `shell` for command execution
- `AnthropicModel`: Interface to Claude models

> 📝 **Note**: These imports provide the core functionality we need. The `editor` and `shell` tools are particularly important as they allow our agent to create files and run commands, which enables the self-extending capability.

In [ ]:
# Import necessary libraries
import os

# Typing for type hints
from typing import Any

# Strands core components
from strands import Agent  # The main agent class

# Tools that allow the agent to interact with the file system and shell
from strands_tools import editor, shell # editor for file operations, shell for command execution

# Claude model integration
from strands.models.anthropic import AnthropicModel

# Hooks for logging tool invocations
from strands.hooks import HookProvider, HookRegistry
from strands.experimental.hooks import BeforeToolInvocationEvent

# Pretty printing for better output formatting
from pprint import pprint

# Color coding for better visibility
from colorama import Fore, Back, Style, init
init(autoreset=True)  # Automatically reset colors after each print

## 3. Configure API Access

We need to set up access to the Anthropic Claude API. This requires an API key that should be stored in your `.env` file.

In [ ]:
# Get Anthropic API key and model ID from environment variables
# These should be defined in your .env file as:
# ANTHROPIC_API_KEY=your_api_key_here
# ANTHROPIC_CLAUDE_4=claude-sonnet-4-20250514 (or your preferred model version)

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
if not ANTHROPIC_API_KEY:
    raise ValueError("Please set the ANTHROPIC_API_KEY environment variable.")

# Default to Claude 4 if not specified in environment variables
ANTHROPIC_MODEL_ID = os.getenv("ANTHROPIC_CLAUDE_4", "claude-sonnet-4-20250514")

## 4. Enable Tool Consent Bypass

For demonstration purposes, we'll bypass the tool consent prompts. In a production environment, you might want to keep this safety feature enabled.

In [ ]:
# Enable auto-approval for the file_write tool
# This bypasses prompts asking for permission when the agent wants to write files
# IMPORTANT: For production use, you may want to disable this for security reasons
os.environ["BYPASS_TOOL_CONSENT"] = "true"

## 5. Define the System Prompt

The system prompt is crucial for guiding the agent's behavior. It explains:
1. What the agent should create (Python tools)
2. Where the tools should be saved (tools directory)
3. How to test the tools (with pytest)
4. The template for tool creation

> 📝 **Note**: The system prompt is critical for self-extending agents. Pay attention to how it:
> - Specifies the directory structure (`cwd()/tools/*.py`)
> - Mentions hot-reloading (so new tools can be used immediately)
> - Includes instructions for testing (creating test files and using pytest)
> - Provides a template for the tool format
>
> You can modify this prompt to change how the agent creates tools.

In [ ]:
# Define the system prompt to guide the agent's behavior
# This prompt is critical as it provides instructions to the agent on:
# 1. What it should create (Python tools)
# 2. Where to store them (tools directory)
# 3. How to test them (unit tests with pytest)
# 4. What to do if tests fail (debug and fix)

SYSTEM_PROMPT = """
You are a specialized agent capable of creating Python tools and using them to solve problems.

Goal:
    - Create Python tools under cwd()/tools/*.py using the tool decorator
    - Each tool should be well-documented with clear docstrings and type hints
    - After writing a tool, you can use it immediately with load_tool (hot-reloading enabled)
    - Create comprehensive unit tests for each tool under cwd()/tools/tests/test_*.py
    - Verify tests work by running them with pytest
    - Debug and fix any issues found during testing

Process for each tool:
    1. Design the tool function with appropriate parameters and return types
    2. Implement the tool with robust error handling and documentation
    3. Create test cases covering normal usage and edge cases
    4. Run tests to verify functionality
    5. Use the tool to solve the specified problem
    6. Document your approach and results

Tool template:
    from strands import tool

    @tool
    def tool_name(param1: type, param2: type) -> return_type:
        '''
        Clear description of what the tool does.
        
        Args:
            param1: Description of parameter 1
            param2: Description of parameter 2
            
        Returns:
            Description of return value
            
        Raises:
            ErrorType: When and why this error might occur
        '''
        # Implementation
        return result

Directory structure:
    - tools/            # Main directory for all tools
      - __init__.py     # Make it a proper package
      - tool_name.py    # One file per tool
      - tests/          # Directory for all tests
        - __init__.py   # Make it a proper package
        - test_tool_name.py  # One test file per tool
"""

## 6. Configure the Anthropic Model

Here we configure the Claude model with appropriate parameters:
- `max_tokens`: Sets the maximum number of tokens in the model's response
- `temperature`: Controls randomness (1.0 is more creative, 0.0 is more deterministic)
- `thinking`: Enables the model to "think" before responding, which helps with complex reasoning

In [ ]:
# Configure Anthropic model
# For self-extending agents, we recommend using Claude 4 with thinking enabled
# The "thinking" parameter allows the model to work through complex problems step by step
anthropic_model = AnthropicModel(
    client_args={
        "api_key": ANTHROPIC_API_KEY
    },
    # Set a generous max tokens limit for complex tool creation tasks
    max_tokens=8000,
    model_id=ANTHROPIC_MODEL_ID,
    params={
        # Temperature of 1.0 encourages more creative responses
        # For more deterministic behavior, you can lower this value
        "temperature": 1,
        
        # Enable "thinking" mode to help the model reason through complex tasks
        # This significantly improves the quality of generated tools
        "thinking": {
            "type": "enabled",
            "budget_tokens": 1028  # Number of tokens allocated for thinking
        }
    }
)

## Hooks

Hooks in Strands Agents provide a powerful event-driven system that allows you to intercept and respond to various events during the agent's execution lifecycle. They enable you to add custom behavior without modifying the core agent logic.

In this notebook, we use a logging hook to monitor tool invocations, which is particularly valuable for debugging and understanding what the self-extending agent is doing:

**Why use hooks for logging tool calls?**
- **Transparency**: See exactly which tools are being called and with what inputs
- **Debugging**: Track the agent's decision-making process when creating and using tools
- **Monitoring**: Observe the agent's behavior during the self-extension process
- **Audit Trail**: Keep a record of all tool operations for analysis

The `BeforeToolInvocationEvent` fires just before any tool is executed, giving us insight into:
1. Which agent is making the call
2. What tool is being invoked
3. What input parameters are being passed

This is especially useful for self-extending agents because you can observe:
- When the agent creates new tools (via the editor tool)
- When it tests tools (via the shell tool)
- When it uses the newly created tools

In [ ]:
# Color-coded logging hook for tool invocations before they are executed
class LoggingHook(HookProvider):
    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(BeforeToolInvocationEvent, self.log_start)

    def log_start(self, event: BeforeToolInvocationEvent) -> None:
        # Color-coded output for better visibility
        print(f"{Fore.CYAN}{'='*60}")
        print(f"{Fore.YELLOW}🔧 TOOL INVOCATION")
        print(f"{Fore.CYAN}{'='*60}")
        print(f"{Fore.GREEN}Agent: {Fore.WHITE}{event.agent.name}")
        print(f"{Fore.BLUE}Tool: {Fore.WHITE}{event.tool_use['name']}")
        print(f"{Fore.MAGENTA}Input Parameters:")
        
        # Pretty print the input with color coding
        import json
        formatted_input = json.dumps(event.tool_use['input'], indent=2)
        for line in formatted_input.split('\n'):
            print(f"{Fore.WHITE}  {line}")
        
        print(f"{Fore.CYAN}{'='*60}")

## 7. Create the Self-Extending Agent

Now we'll create the agent with our system prompt, model, and tools:
- `system_prompt`: Instructions we defined earlier
- `model`: The configured Anthropic model
- `tools`: The editor and shell tools that allow file creation and command execution
- `load_tools_from_directory`: Enables hot-reloading of tools, so the agent can use tools it just created

> 📝 **Note**: The `load_tools_from_directory=True` parameter is essential for self-extending functionality. Without this, the agent wouldn't be able to use the tools it creates without restarting the notebook.

In [ ]:
# Create self-extending agent with Anthropic Claude 4
# This agent will be able to:
# 1. Create new Python tools
# 2. Test these tools
# 3. Debug and fix any issues
# 4. Use the tools immediately after creation

agent = Agent(
    system_prompt=SYSTEM_PROMPT,  # The instructions we defined earlier
    model=anthropic_model,        # The Claude 4 model we configured
    tools=[editor, shell],        # Tools for file operations and shell commands
    hooks=[LoggingHook()],        # Hooks for logging tool invocations
    
    # This is critical for self-extending functionality:
    # When set to True, the agent can dynamically load tools it creates
    # without restarting the runtime
    load_tools_from_directory=True
)

In [ ]:
# Color-coded agent response wrapper
def run_agent_with_colors(agent, prompt):
    """
    Run the agent with a prompt and display the response with color coding.
    This makes it easier to distinguish between different types of output.
    """
    print(f"{Fore.GREEN}{'='*80}")
    print(f"{Fore.YELLOW}🤖 AGENT REQUEST")
    print(f"{Fore.GREEN}{'='*80}")
    print(f"{Fore.CYAN}Prompt: {Fore.WHITE}{prompt}")
    print(f"{Fore.GREEN}{'='*80}")
    print(f"{Fore.YELLOW}📝 Agent Response:")
    print(f"{Fore.GREEN}{'='*80}")
    
    # Run the agent and capture the result
    result = agent(prompt)
    
    print(f"{Fore.GREEN}{'='*80}")
    print(f"{Fore.YELLOW}✅ AGENT RESPONSE COMPLETE")
    print(f"{Fore.GREEN}{'='*80}")
    
    return result


## Color Coding Features

This notebook now includes color-coded output to make it easier to distinguish between different types of information:

- **🔧 Tool Invocations**: Cyan borders with yellow headers show when tools are being called
- **🤖 Agent Requests**: Green borders with yellow headers for agent prompts
- **📝 Agent Responses**: Clear separation between requests and responses
- **✅ Completion Indicators**: Visual confirmation when operations complete

The color coding helps you:
- **Track Progress**: See exactly which tools are being invoked and when
- **Debug Issues**: Quickly identify where problems occur in the tool chain
- **Understand Flow**: Follow the agent's decision-making process step by step
- **Monitor Performance**: See how long different operations take


In [ ]:
# Test the color coding system
print("Testing color coding system...")
print(f"{Fore.RED}Red text for errors")
print(f"{Fore.GREEN}Green text for success")
print(f"{Fore.YELLOW}Yellow text for warnings")
print(f"{Fore.BLUE}Blue text for information")
print(f"{Fore.MAGENTA}Magenta text for highlights")
print(f"{Fore.CYAN}Cyan text for separators")
print(f"{Fore.WHITE}White text for normal content")
print(f"{Fore.GREEN}{'='*50}")
print(f"{Fore.YELLOW}Color coding test complete!")
print(f"{Fore.GREEN}{'='*50}")


## 8. Interact with the Self-Extending Agent

Let's define a prompt that asks the agent to create and use multiple tools to solve different problems:

> 📝 **Note**: When creating your own prompts, be specific about the tools you need and the problems you want solved. Include clear examples and expected outputs.

In [ ]:
# Define the prompt for the agent
# This prompt asks the agent to create multiple tools and solve specific problems
# It includes clear expectations for testing and demonstration

prompt = """
I need you to create several Python tools to solve the following problems:

1. Create a tool named 'add_numbers' that adds two numbers, then use it to add 5 and 7.

2. Create a tool named 'calculate_compound_interest' that calculates compound interest with parameters for principal, rate, time, and compounding frequency. Use it to find the final value of $1000 invested at 5% annual interest for 3 years, compounded quarterly.

3. Create a tool named 'analyze_text' that takes a text string and returns statistics including: word count, character count, average word length, and most frequent word. Use it to analyze this sample text: "The quick brown fox jumps over the lazy dog. The fox was quick and brown and very clever."

For each tool:
1. Create the tool in the proper directory
2. Create comprehensive unit tests
3. Run the tests using pytest and ensure they pass
4. Show the tool in action by solving the specified problem
5. Provide the exact result of using the tool

After creating all tools, summarize which tools you've created, which tests were run, and all the results obtained.
"""

## 9. Run the Agent to Create Tools

Now we'll run the agent with our prompt. This will:
1. Trigger the agent to understand the task
2. Create necessary directories if they don't exist
3. Write the Python tool files
4. Create test files for the tools
5. Use the tools to demonstrate their functionality

> 📝 **Note**: This cell may take several minutes to run as the agent thinks through the implementation, writes files, and tests them.

In [ ]:
# Invoke the agent with our prompt using color-coded output
# This will trigger the agent to:
# 1. Create directories if needed (tools/ and tools/tests/)
# 2. Write Python tool files with the requested functionality
# 3. Create test files to verify the tools work
# 4. Execute the tools to demonstrate their functionality
#
# Note: This cell may take several minutes to run as the agent thinks through 
# the implementation, writes files, and tests them

result = run_agent_with_colors(agent, prompt)

## 10. Testing and Debugging the Created Tools

After creating the tools, we need to ensure they work correctly. We'll ask the agent to:
1. Run the tests it created
2. Debug any issues it encounters
3. Fix problems in the implementation

> 📝 **Note**: This is where we demonstrate the interactive development process. By asking the agent to run tests, we show how you can guide it through a complete development cycle.

In [ ]:
# Ask the agent to explicitly run tests on the created tools
# This demonstrates how you can guide the agent through a complete development process
# Including testing to ensure quality and functionality
test_prompt = "Did you run all the tests for the tools you created? Please verify that all tests pass and show me the results."
run_agent_with_colors(agent, test_prompt)

## 11. Examine Agent Conversation History

We can inspect the agent's message history to understand how it approached the task. This helps us see the agent's reasoning process and how it handled any challenges.

In [ ]:
# View the full conversation history with the agent
# This shows the entire interaction, including the agent's thinking process
# Analyzing this can provide insights into how the agent approaches tool creation
agent.messages

## 12. Review Tool Invocations

Finally, we can ask the agent to report which tools it called during the process. This helps us understand how the agent used its available tools to accomplish the task.

In [ ]:
# Ask the agent to list all the tools it has called during this session
# This helps understand which built-in and newly created tools were used
# It's a good way to verify that the agent is actually using the tools it created
summary_prompt = "Please list all the tools you've created and used during this session, and summarize what each tool does and the results you obtained when using them."
run_agent_with_colors(agent, summary_prompt)

## Expected Output and Results

When running this notebook, you can expect to see the following results:

### Tool Creation Phase
When the agent runs for the first time, you'll see output showing:

1. Directory creation (if needed):
   ```
   I'll create the necessary directories for our tools and tests.
   ```

2. Tool implementation for adding numbers:
   ```python
   # tools/add_numbers.py
   from strands import tool

   @tool
   def add_numbers(a: float, b: float) -> float:
       """
       Add two numbers together.
       
       Args:
           a: First number
           b: Second number
           
       Returns:
           Sum of the two numbers
       """
       return a + b
   ```

3. Test file creation:
   ```python
   # tools/tests/test_add_numbers.py
   import pytest
   from tools.add_numbers import add_numbers

   def test_add_numbers():
       assert add_numbers(5, 7) == 12
       assert add_numbers(-3, 5) == 2
       assert add_numbers(0, 0) == 0
       assert add_numbers(1.5, 2.5) == 4.0
   ```

4. Tool demonstration (using the created tool):
   ```
   Let me use the add_numbers tool to add 5 and 7:
   Result: 12
   ```

5. Similar output for the compound interest and text analysis tools.

### Testing Phase
When running tests, you'll see output like:
```
Running pytest to verify our tests...
============================= test session starts ==============================
...
tools/tests/test_add_numbers.py ..
tools/tests/test_compound_interest.py ..
tools/tests/test_analyze_text.py ..
============================== 6 passed in 0.15s ==============================
```

## 13. Conclusion and Next Steps

In this notebook, we've demonstrated how to create a self-extending agent that can:
1. Generate new tools based on natural language descriptions
2. Test and debug its own creations
3. Immediately use the tools it creates

This capability is powerful for creating adaptive AI systems that can expand their functionality in response to user needs.

### Possible Extensions

- Create more complex tools that interact with APIs or databases
- Build tools that can modify or improve existing tools
- Create a pipeline of tools that work together for complex workflows
- Implement version control for generated tools
- Add a feedback mechanism to improve tool quality over time

### Troubleshooting Tips

If you encounter issues:

1. **Missing directories**: Ensure the agent creates the `tools` and `tools/tests` directories
2. **Import errors**: Check if the necessary packages are installed
3. **Test failures**: Examine the test output and ask the agent to debug
4. **Permission issues**: Make sure your environment allows file writing and shell execution
5. **API limits**: Watch for token limits if tools require complex implementations
6. **Missing __init__.py files**: If you get import errors, check that the agent created the necessary __init__.py files to make the directories proper Python packages